# Goal

The goal of this demo is to demonstrate how the RandomNearIncumbentOptimizer works.

# Method

1. We create a 2-parameter single-objective synthetic function for the BayesianOptimizer to maximize.
2. We either use the pass-through model or train the initial model on a bunch of random parameters.
3. We plot:
    1. The original function
    2. The models predictions
    3. The utility function values
    
4. We create the RandomNearIncumbentTracer and subscribe to all RandomNearIncumbentOptimizer events.
5. For each call to .suggest():
    1. We capture the initial incumbents and their utility values.
    2. For each call to _run_iteration():
        1. We capture the random neighbors and their utility values.
        2. We capture the new incumbents (and maybe even draw an arrow??)

6. We visualize all the data captured in 5. 

So for each scene we will need:
    1. Optionally the true objective function surface plot.
    2. Utility function surface plot.
    3. All past incumbents 3D scatter plot.
    5. All current incumbents 3D scatter plot.
    4. All current neighbors 3D scatter plot.